In [21]:
##Let's get started

import pandas as pd
import requests
import urllib.parse
import time

df2020 = pd.read_csv("dfc_2020_geocoded_filter.csv")
df2025 = pd.read_csv("dfc_2025_geocoded_filter.csv")

df2020.shape
df2020.info()
df2020.dtypes

df2025.shape
df2025.info()
df2025.dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7578 entries, 0 to 7577
Data columns (total 17 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   CCN                               7578 non-null   int64  
 1   NETWORK                           7578 non-null   int64  
 2   Facility Name                     7578 non-null   object 
 3   Five Star Date                    7578 non-null   object 
 4   Five Star                         7578 non-null   object 
 5   Five Star Data Availability Code  7578 non-null   int64  
 6   Address Line 1                    7578 non-null   object 
 7   Address Line 2                    1083 non-null   object 
 8   City                              7578 non-null   object 
 9   State                             7578 non-null   object 
 10  Zip                               7578 non-null   int64  
 11  County                            7578 non-null   object 
 12  dfc_cl

CCN                                   int64
Network                               int64
Facility Name                        object
Five Star Date                       object
Five Star                           float64
Five Star Data Availability Code      int64
Address Line 1                       object
Address Line 2                       object
City                                 object
State                                object
Zip                                   int64
County/Parish                        object
dfc_new                              object
full_address                         object
latitude                            float64
longitude                           float64
dtype: object

In [22]:
##convert column headings so everything is consistent and remove whitespace and other inconsistencies

#Make all column headers lower case because I don't do these fancy capital letters
df2020.columns = df2020.columns.str.lower()
df2025.columns = df2025.columns.str.lower()

#Make 2025 column headings match 2020
df2025.rename(columns={"county/parish": "county"}, inplace=True)

#rename weird 2020 column heading
df2020.rename(columns={"dfc_closed.1": "dfc_closed"}, inplace=True)

#strip any extraneous whitespace from county and state columns
df2020['county'] = df2020['county'].str.strip()
df2020['state'] = df2020['state'].str.strip()

df2025['county'] = df2025['county'].str.strip()
df2025['state'] = df2025['state'].str.strip()

##create combined county/state variable

df2020['county_state'] = df2020['county'] + ", " + df2020['state']
df2025['county_state'] = df2025['county'] + ", " + df2025['state']

#double checking everything worked
df2020.info()
df2020.dtypes

df2025.info()
df2025.dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7578 entries, 0 to 7577
Data columns (total 18 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   ccn                               7578 non-null   int64  
 1   network                           7578 non-null   int64  
 2   facility name                     7578 non-null   object 
 3   five star date                    7578 non-null   object 
 4   five star                         7578 non-null   object 
 5   five star data availability code  7578 non-null   int64  
 6   address line 1                    7578 non-null   object 
 7   address line 2                    1083 non-null   object 
 8   city                              7578 non-null   object 
 9   state                             7578 non-null   object 
 10  zip                               7578 non-null   int64  
 11  county                            7578 non-null   object 
 12  dfc_cl

ccn                                   int64
network                               int64
facility name                        object
five star date                       object
five star                           float64
five star data availability code      int64
address line 1                       object
address line 2                       object
city                                 object
state                                object
zip                                   int64
county                               object
dfc_new                              object
full_address                         object
latitude                            float64
longitude                           float64
county_state                         object
dtype: object

In [23]:
#create list of counties with clinics
counties_with_clinics = set(df2025['county_state'])
counties_with_clinics_2020 = set(df2020['county_state'])

In [24]:
#import full county list

counties = pd.read_csv("demographic_data.csv")

counties.shape
counties.info()
counties.dtypes



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3222 entries, 0 to 3221
Data columns (total 12 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   County                           3222 non-null   object
 1   State                            3222 non-null   object
 2   State FIPS Code                  3222 non-null   int64 
 3   County FIPS Code                 3222 non-null   int64 
 4   FIPS                             3222 non-null   int64 
 5   Total Population                 3222 non-null   int64 
 6   Male Population                  3222 non-null   int64 
 7   Female Population                3222 non-null   int64 
 8   Total Race Responses             3222 non-null   int64 
 9   White Alone                      3222 non-null   int64 
 10  Black or African American Alone  3222 non-null   int64 
 11  Hispanic or Latino               3222 non-null   int64 
dtypes: int64(10), object(2)
memory usa

County                             object
State                              object
State FIPS Code                     int64
County FIPS Code                    int64
FIPS                                int64
Total Population                    int64
Male Population                     int64
Female Population                   int64
Total Race Responses                int64
White Alone                         int64
Black or African American Alone     int64
Hispanic or Latino                  int64
dtype: object

In [2]:
#Make all column headers lower case because I don't do these fancy capital letters
counties.columns = counties.columns.str.lower()

#strip extraneous whitespace
counties['county'] = counties['county'].str.strip()
counties['state'] = counties['state'].str.strip()

#make a combined county/state variable
counties['county_state'] = counties['county'] + ", " + counties['state']

#double check everything worked
counties.info()
counties.dtypes

NameError: name 'counties' is not defined

In [1]:
#create list of counties WITH and WITHOUT dialysis clinics

# Mark counties WITHOUT a clinic
counties['with_clinic'] = counties['county_state'].isin(counties_with_clinics)

# Filter for counties with no clinic
df_no_clinic = counties[~counties['with_clinic']]

NameError: name 'counties' is not defined

In [4]:
#import geoJSON file of US counties

import geopandas as gpd

#convert json from latin-1 encoding to utf-8

with open("us_counties.json", "r", encoding="latin-1") as f:
    data = f.read()

with open("us_counties_utf8.json", "w", encoding="utf-8") as f:
    f.write(data)

gdf = gpd.read_file("us_counties_utf8.json")

#make everything lowercase
gdf.columns = gdf.columns.str.lower()

#states are listed by FIPS code. Convert to standard postal abbreviations
state_fips_to_abbr = {
    '01': 'AL', '02': 'AK', '04': 'AZ', '05': 'AR', '06': 'CA',
    '08': 'CO', '09': 'CT', '10': 'DE', '11': 'DC', '12': 'FL',
    '13': 'GA', '15': 'HI', '16': 'ID', '17': 'IL', '18': 'IN',
    '19': 'IA', '20': 'KS', '21': 'KY', '22': 'LA', '23': 'ME',
    '24': 'MD', '25': 'MA', '26': 'MI', '27': 'MN', '28': 'MS',
    '29': 'MO', '30': 'MT', '31': 'NE', '32': 'NV', '33': 'NH',
    '34': 'NJ', '35': 'NM', '36': 'NY', '37': 'NC', '38': 'ND',
    '39': 'OH', '40': 'OK', '41': 'OR', '42': 'PA', '44': 'RI',
    '45': 'SC', '46': 'SD', '47': 'TN', '48': 'TX', '49': 'UT',
    '50': 'VT', '51': 'VA', '53': 'WA', '54': 'WV', '55': 'WI',
    '56': 'WY', '60': 'AS', '66': 'GU', '69': 'MP', '72': 'PR',
    '78': 'VI'
}

# Convert FIPS to strings with leading zeros if needed
gdf['state'] = gdf['state'].astype(str).str.zfill(2)

# Create a new column with state abbreviations
gdf['state_postal'] = gdf['state'].map(state_fips_to_abbr)

#create new column with county/state
gdf['county_state'] = gdf['name'] + ", " + gdf['state_postal']

#check column headers
print(gdf.head())

           geo_id state county       name    lsad  censusarea  \
0  0500000US01029    01    029   Cleburne  County     560.100   
1  0500000US01031    01    031     Coffee  County     678.972   
2  0500000US01037    01    037      Coosa  County     650.926   
3  0500000US01039    01    039  Covington  County    1030.456   
4  0500000US01041    01    041   Crenshaw  County     608.840   

                                            geometry state_postal  \
0  POLYGON ((-85.38872 33.91304, -85.38088 33.873...           AL   
1  POLYGON ((-86.03044 31.61894, -86.00408 31.619...           AL   
2  POLYGON ((-86.00928 33.10164, -86.00917 33.090...           AL   
3  POLYGON ((-86.34851 30.99434, -86.35023 30.994...           AL   
4  POLYGON ((-86.14699 31.68046, -86.14711 31.663...           AL   

    county_state  
0   Cleburne, AL  
1     Coffee, AL  
2      Coosa, AL  
3  Covington, AL  
4   Crenshaw, AL  


In [3]:
# Create layer of all counties with clinics in 2020
gdf_2020 = gdf[gdf['county_state'].isin(counties_2020)]

# Create layer of counties that lost clinics
gdf_closed = gdf[gdf['county_state'].isin(closed_counties)]

# save other layers to file
gdf_2020.to_file("counties_with_clinics_2020.geojson", driver="GeoJSON")
gdf_closed.to_file("counties_lost_clinics.geojson", driver="GeoJSON")

NameError: name 'gdf' is not defined